In [1]:
import pandas as pd

%env GOOGLE_APPLICATION_CREDENTIALS=/Users/mike/.gcp-keys/pokemechs-84c9b2aac2f8.json

env: GOOGLE_APPLICATION_CREDENTIALS=/Users/mike/.gcp-keys/pokemechs-84c9b2aac2f8.json


In [2]:

from typing import List, Optional

from vertexai.language_models import TextEmbeddingInput, TextEmbeddingModel



In [3]:
def embed_text(
    texts: List[str],
    task: str = "RETRIEVAL_DOCUMENT",
    model_name: str = "text-embedding-004",
    dimensionality: Optional[int] = 256,
) -> List[List[float]]:
    """Embeds texts with a pre-trained, foundational model."""
    model = TextEmbeddingModel.from_pretrained(model_name)
    inputs = [TextEmbeddingInput(text, task) for text in texts]
    kwargs = dict(output_dimensionality=dimensionality) if dimensionality else {}
    embeddings = model.get_embeddings(inputs, **kwargs)
    return [embedding.values for embedding in embeddings]


In [76]:

filename = './manuals/Subaru-outback-starter-and-generater/Subaru_Legacy_Outback_starter-generater.md'
with open(filename, 'r') as fd:
    text = fd.read()



In [ ]:
split_texts = text.split('\n## ')

In [ ]:

split_text_with_identifier = []

for i in split_texts:
    text_identifier = "Alternator/Starter"
    modified_text = f"## {text_identifier}\n{i}"
    split_text_with_identifier.append(modified_text)


In [67]:
embedded_test = embed_text([text])

In [68]:
import json


output_filename = './manuals/embedded/Subaru_Outback.json'

with open(output_filename, 'w') as f:
    json.dump([list(emb) for emb in embedded_test], f) 


In [8]:
import psycopg2

In [9]:
psycopg2.connect

<function psycopg2.connect(dsn=None, connection_factory=None, cursor_factory=None, **kwargs)>

In [10]:
con =psycopg2.connect("user=postgres password=postgres host=127.0.0.1 port=64322 dbname=postgres")



In [93]:
yearmin = 2010
yearmax = 2013
# with con.cursor() as cur:
#     cur.execute("insert into subaru (name, yearmin, yearmax, embedding, text) values ('Legacy-Outback starter/generater', %s, %s, %s, %s)", (yearmin, yearmax, embedded_test[0], text))
#     con.commit()
for i in split_text_with_identifier:
    embedded_text = embed_text([i])
    with con.cursor() as cur:
        cur.execute("insert into subaru (name, yearmin, yearmax, embedding, text) values ('Legacy-Outback starter/generater', %s, %s, %s, %s)", (yearmin, yearmax, embedded_text[0], i))
        con.commit()


In [97]:

def search(query, topk=5):
    try:
        dimensionality: Optional[int] = 256
        model = TextEmbeddingModel.from_pretrained("text-embedding-004") 
        query_embedding = model.get_embeddings([TextEmbeddingInput(query, "RETRIEVAL_QUERY")], output_dimensionality=dimensionality)[0].values
        print(len(query_embedding))

        with con.cursor() as cur:

            cur.execute(
                """
                SELECT name, text, 1 - (embedding <-> %s::vector(256)) AS similarity 
                FROM subaru
                ORDER BY similarity DESC
                LIMIT %s
                """,
                (query_embedding, topk)
            )
            results = cur.fetchall()
    except Exception as e:
        print(f"Error executing query: {e}")
        con.rollback()
        return []
    else:
        con.commit()
    finally:
        cur.close() 

    return results

In [105]:
search_results = search("remove generator")
for result in search_results:
    print(f"Name: {result[0]}, Similarity: {result[2]}\nText: {result[1]}\n---")

prompt = f"""

Hey you are a n ai assistent here is some sections from a manual

Content
---
 {search_results}
---
"""



256
Name: Legacy-Outback starter/generater, Similarity: 0.47078266248731
Text: ## Alternator/Starter
3. Generator A:  Removal

![16_image_0.png](16_image_0.png)

BELT,      REMOVAL,
V-belt.>
to ME(H4DOTC)-42, V-BELT, REMOVAL, V-belt.>
<Ref. to ME(H6DO)-51, REMOVAL, V-belt.> 3) Disconnect the connector and terminal from

gnnrjter.

![16_image_2.png](16_image_2.png)

4) Remove the collector cover bracket or v-belt
cover bracket.

![16_image_5.png](16_image_5.png)

![16_image_6.png](16_image_6.png)

---
Name: Legacy-Outback starter/generater, Similarity: 0.417108672004022
Text: ## Alternator/Starter
1.  2.5 L Model

1) Remove the four through-bolts.

![16_image_3.png](16_image_3.png)

2) Use a drier to heat the rear cover (A) portion to 50°C (122°F).

![16_image_4.png](16_image_4.png)

3) Insert the end of a flat tip screwdriver into the gap between stator core and front cover. Pry them

apart to disassemble.

![17_image_0.png](17_image_0.png)

(A) Screwdriver 4) Using a vise, support the